In [1]:
import sys
sys.path.append('../')
import numpy as np
from algo_trading.dataset import *
from algo_trading.analytics.metrics import *

In [2]:
financial_summary_data = load_financial_summary_data()

In [3]:
stock_price_data = load_stock_price_data()

## 최근 3개년 ROE 증가 추세에 있는 종목

In [4]:
roe_increasing_stocks = []

for stock_code, fin_data in financial_summary_data.items():
    try:
        if check_metric_increasing(fin_data['annual'], start_year=2017, end_year=2019,
                                   metric='ROE', should_be_positive=True):
            roe_increasing_stocks.append(stock_code)
    except:
        continue

### RIM 모델 통해 적정 주가 계산

In [5]:
stock_code_list = []
proper_price_list = []
current_price_list = []

for stock_code, fin_data in financial_summary_data.items():
    try:
        proper_price = rim_price(fin_data['annual'], 2019)
        current_price = int(stock_price_data[stock_code].iloc[-1]['Close'])
    except:
        continue
    stock_code_list.append(stock_code)
    proper_price_list.append(proper_price)
    current_price_list.append(current_price)

In [6]:
rim_df = pd.DataFrame({'종목코드':stock_code_list,
                       '적정주가':proper_price_list,
                       '현재주가':current_price_list})

## 적정 주가 대비, 현재 주가가 3분의 1 수준 미만인 종목

In [7]:
low_price_df = rim_df.loc[rim_df['적정주가'] > rim_df['현재주가'] * 3]

In [8]:
candidate_stock_codes = set(low_price_df['종목코드']) & set(roe_increasing_stocks)

In [9]:
rim_df.loc[rim_df['종목코드'].apply(lambda x: x in candidate_stock_codes)]

,종목코드,적정주가,현재주가
87,001500,30262,9150
229,005670,28207,5280
274,007340,80051,23650
843,003070,33549,10200
1057,002870,5574,1755
1085,019180,9834,2635
1156,101330,19489,3435
1501,138930,17885,5080
1591,003530,5668,1750
1762,175330,19486,4770
